# Web Scraping for Used Car Prices from Autotrader.ca

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing required modules
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import re
import pymongo
import pandas as pd
from sqlalchemy import create_engine,inspect, func
import numpy as np
import time
#from config import api_key

In [13]:
# This is to initialize Splinter for Windows users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
# list to store extracted values

title_list=[]
year_list=[]
price_list=[]
loc_list=[]
date_list=[]
milage_list=[]
car_list=[]
model_list=[]
trans_list=[]

# loop to scrap website

url = "https://www.autotrader.ca/cars/?rcp=15&rcs=0&srt=9&prx=-1&hprc=True&wcp=True&sts=New-Used&inMarket=advancedSearch"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_='newSRPDesign')


i=1

while i<=2:
  
    for result in results:
       
        
        try:
            
            # scrape the price
            price = result.find('span', class_='price-amount').text
            price_list.append(price)
            # scrape the  Title
            ctitle = result.find('div', class_='listing-details').h2.a.span.text
            #words=['lease', 'take over', 'finance', 'financing', 'leasing', 'takeover', 'financed', 'leased', 'transfer', 'transferring']
            #for word in words:
            
                #if word in result.find('div', class_='title').text.split():
                    #print('a listing including lease or finance was removed')
                #elif word in result.find('div', class_='description').text.split():
                    #print('a listing including lease or finance was removed')
                #else:
            btitle= ctitle.split('\n')[1].replace('                                    ','')
            title = btitle.split('                                    ')[0]
            year = title.split(" ")[0]
            cartype= title.split(" ")[1]
            model = title.split(" ")[2]
            title_list.append(title)
            year_list.append(year)
            car_list.append(cartype)
            model_list.append(model)
         
            
            
        # scrape the  Location
            #location = result.find('div', class_='location').text.split()[0]
            #loc_list.append(location)
        
         
        
        
        
        # scrape the  Date
            #date = result.find('span', class_='date-posted').text
            #date_list.append(date)
        # scrape the  details
            details = result.find('div', class_='kms').text#.split('"')[0]         
            milage=details
           
            
            milage_list.append(milage)            
   
    
        except AttributeError:
            date=''
        except IndexError:
            cartype=''
            model=''
    
    browser.find_link_by_text(1+i).first.click()
    i=i+1    
# store the extracted valuse in dictionary            
Dict={}
Dict["Title"]=title_list
Dict["Car Type"]=car_list
Dict["Model"]=model_list
Dict["Price"]=price_list
#Dict["Location"]=loc_list
#Dict["Date"]=date_list
Dict["Milage"]=milage_list
Dict["Year"]=year_list
#Dict["Transmission"]=trans_list


    



In [16]:
df=pd.DataFrame.from_dict(Dict, orient='index')
df=df.transpose()
df

,Title,Car Type,Model,Price,Milage,Year
0,2020 Nissan Murano PLATINUM,Nissan,Murano,"$44,613","\nMileage 36,447 km\n ...",2020
1,2017 Lexus RX 350 Executive Pkg- Panoramic Roo...,Lexus,RX,"$37,900","\nMileage 162,300 km\n ...",2017
2,2011 BMW 328 X-DRIVE 6 VITESSE MANUEL CUIR TOI...,BMW,328,"$6,994","\nMileage 81,736 km\n ...",2011
3,"2015 Mazda Mazda3 GS W/SUNROOF, LOW KMS, AUTO,...",Mazda,Mazda3,"$12,988","\nMileage 9,058 km\n ...",2015
4,2019 Volkswagen Atlas COMFORTLINE,Volkswagen,Atlas,"$47,749","\nMileage 41,105 km\n ...",2019
5,2019 Toyota Corolla XSE: Sunroof/ Navigation/ ...,Toyota,Corolla,"$21,900","\nMileage 124,000 km\n ...",2019
6,2011 Audi A3 TDI Premium,Audi,A3,"$10,995","\nMileage 8,310 km\n ...",2011
7,2019 BMW i8 Coupe,BMW,i8,"$102,900","\nMileage 42,132 km\n ...",2019
8,2019 Dodge Grand Caravan CANADA VALUE PKG*REAR...,Dodge,Grand,"$20,844","\nMileage 60,243 km\n ...",2019
9,2018 Hyundai Tucson 2.0L SE AWD,Hyundai,Tucson,"$26,888","\nMileage 83,787 km\n ...",2018


In [29]:
df['Title'].value_counts()

2018 Toyota Prius v GAS SAVING HYBRID-EDITION 5 PASSENGER 1.8L - DOHC.    4
2017 Ram 2500 Mega Cab Limited                                            2
2016 Honda Civic lx                                                       2
2016 Audi A4 Komfort plus                                                 2
2012 Chevrolet Corvette Grand Sport                                       2
1974 Ford Bronco                                                          2
2020 Chevrolet Suburban LT  - Leather Seats - Luxury Package              2
2019 Honda Fit AUTOMATIQUE                                                2
2001 Ford Escape 4dr 103  WB XLS 4WD                                      2
2017 Ford F-150 4WD SuperCrew 145 XLT                                     2
2018 BMW 3 Series 330i xDrive M-SPORT *NAVI/CAMERA/ROOF/RED INT/LED       2
2008 Ford Focus 2dr Cpe SES                                               2
2018 Dodge Journey SXT  - Low Mileage                                     2
1974 Jaguar 

In [ ]:
df["Price"] = df["Price"].str.replace("$", "")
df["Price"] = df["Price"].str.replace(",", "")
df["Milage"] = df["Milage"].str.replace(",", "")
df["Milage"] = df["Milage"].str.replace("\nMileage", "")
df["Milage"] = df["Milage"].str.replace("\n", "")
df["Milage"] = df["Milage"].str.replace("km", "")

In [ ]:
df.reset_index()
df.dropna()
df.drop_duplicates()
df = df.mask(df.eq('None')).dropna()

In [ ]:
df

In [ ]:
df['Milage'].value_count()

In [ ]:
from config import password
connection_string = "postgres:"+password+"@localhost:5432/hondadb"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
#engine.table_names()
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
inspector.get_columns("cardb")

In [ ]:
df.to_sql(name='cardb', con=engine, if_exists='replace', index=True)

In [ ]:
sql_DF = pd.read_sql("SELECT * FROM cardb",
                     con=engine)

In [ ]:
sql_DF.info()